# **Deutsch-Jozsa with Cirq**

In [1]:
pip install cirq

     |████████████████████████████████| 437 kB 52 kB/s s eta 0:00:01
     |████████████████████████████████| 328 kB 3.0 kB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 10 kB/s s eta 0:00:01
     |████████████████████████████████| 55 kB 38 kB/s s eta 0:00:01
     |████████████████████████████████| 47 kB 23 kB/s s eta 0:00:01
     |████████████████████████████████| 93 kB 10 kB/s s eta 0:00:01
     |████████████████████████████████| 13.7 MB 13 kB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 13 kB/s s eta 0:00:01
     |████████████████████████████████| 97 kB 45 kB/s s eta 0:00:01
     |████████████████████████████████| 220 kB 44 kB/s s eta 0:00:01
     |████████████████████████████████| 58 kB 80 kB/s s eta 0:00:01
     |████████████████████████████████| 145 kB 80.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 55 kB/s s eta 0:00:01
     |████████████████████████████████| 53 kB 14 kB/s s eta 0:00:01
     |████████████████████████████████| 65

In [2]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
print(cirq.google.Bristlecone)

                                             (0, 5)────(0, 6)
                                             │         │
                                             │         │
                                    (1, 4)───(1, 5)────(1, 6)────(1, 7)
                                    │        │         │         │
                                    │        │         │         │
                           (2, 3)───(2, 4)───(2, 5)────(2, 6)────(2, 7)───(2, 8)
                           │        │        │         │         │        │
                           │        │        │         │         │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)────(3, 6)────(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │         │         │        │        │
                  │        │        │        │         │         │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)────(4, 6)────(4, 7)───(4, 8)───(4, 9)───(4, 10)
         │        │      

/tmp/ipykernel_361/1631590671.py:1: DeprecationWarning: cirq.google was used but is deprecated.
 it will be removed in cirq v0.14.
 Use cirq_google instead.

  print(cirq.google.Bristlecone)


The following code gives the operations for these functions where we take two input qubits and compute the function in the third qubit.

In [4]:
import warnings
warnings.filterwarnings('ignore')

"""Operations to query all possible functions on two bits.
Two of these functions are constant, and six of these functions are balanced.
"""
# Define three qubits to use.
q0, q1, q2 = cirq.LineQubit.range(3)

# Define the operations to query each of the two constant functions.
constant = (
    [], 
    [cirq.X(q2)]
)

# Define the operations to query each of the six balanced functions.
balanced = (
    [cirq.CNOT(q0, q2)], 
    [cirq.CNOT(q1, q2)], 
    [cirq.CNOT(q0, q2), cirq.CNOT(q1, q2)],
    [cirq.CNOT(q0, q2), cirq.X(q2)], 
    [cirq.CNOT(q1, q2), cirq.X(q2)], 
    [cirq.CNOT(q0, q2), cirq.CNOT(q1, q2), cirq.X(q2)]
)

In [5]:
# Define a function

def deJo_circuit(oracle):
    # Phase kickback trick.
    yield cirq.X(q2), cirq.H(q2)

    # Get an equal superposition over input bits.
    yield cirq.H(q0), cirq.H(q1)

    # Query the function.
    yield oracle

    # Use interference to get result, put last qubit into |1>.
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2)

    # Use a final OR gate to put result in final qubit.
    yield cirq.X(q0), cirq.X(q1), cirq.CCX(q0, q1, q2)
    yield cirq.measure(q2)

In [6]:
"""Verify """
simulator = cirq.Simulator()

print("\nResult on constant functions:")
for oracle in constant:
    result = simulator.run(cirq.Circuit(deJo_circuit(oracle)), repetitions=10)
    print(result)

print("\nResult on balanced functions:")
for oracle in balanced:
    result = simulator.run(cirq.Circuit(deJo_circuit(oracle)), repetitions=10)
    print(result)


Result on constant functions:
2=0000000000
2=0000000000

Result on balanced functions:
2=1111111111
2=1111111111
2=1111111111
2=1111111111
2=1111111111
2=1111111111
